In [ ]:
!unzip "/NewDatasetsForKeras.zip"

Archive:  /content/NewDatasetsForKeras.zip
   creating: validation/
   creating: validation/kursi01/
  inflating: validation/kursi01/kursi01.jpg  
  inflating: validation/kursi01/kursi02.jpg  
  inflating: validation/kursi01/kursi03.jpg  
  inflating: validation/kursi01/kursi04.jpg  
  inflating: validation/kursi01/kursi76.jpg  
  inflating: validation/kursi01/kursi77.jpg  
  inflating: validation/kursi01/kursi78.jpg  
  inflating: validation/kursi01/kursi79.jpg  
  inflating: validation/kursi01/kursi80.jpg  
  inflating: validation/kursi01/kursi81.jpg  
   creating: validation/kursi02/
  inflating: validation/kursi02/kursi05.jpg  
  inflating: validation/kursi02/kursi06.jpg  
  inflating: validation/kursi02/kursi07.jpg  
  inflating: validation/kursi02/kursi08.jpg  
   creating: validation/kursi03/
  inflating: validation/kursi03/kursi09.jpg  
  inflating: validation/kursi03/kursi10.jpg  
  inflating: validation/kursi03/kursi11.jpg  
  inflating: validation/kursi03/kursi12.jpg  
  inf

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameters
img_height, img_width = 224, 224
batch_size = 32

# Training data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validate data generator
val_datagen = ImageDataGenerator(rescale=1./255)

# Load the training data
train_generator = train_datagen.flow_from_directory(
    '/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the validation data
validation_generator = val_datagen.flow_from_directory(
    '/validation',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 122 images belonging to 14 classes.
Found 122 images belonging to 14 classes.


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Load VGG16 model + higher level layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of VGG16 base model
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(14, activation='softmax')(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=150
)

# Save model
model.save('furniture.h5')

Epoch 1/150
3/3 [==============================] - 13s 792ms/step - loss: 6.0097 - accuracy: 0.0833 - val_loss: 3.3537 - val_accuracy: 0.3750
Epoch 2/150
3/3 [==============================] - 7s 3s/step - loss: 6.2358 - accuracy: 0.1444 - val_loss: 1.6959 - val_accuracy: 0.6250
Epoch 3/150
3/3 [==============================] - 2s 673ms/step - loss: 4.3992 - accuracy: 0.2444 - val_loss: 1.1020 - val_accuracy: 0.7292
Epoch 4/150
3/3 [==============================] - 2s 863ms/step - loss: 2.0714 - accuracy: 0.4375 - val_loss: 0.5652 - val_accuracy: 0.8438
Epoch 5/150
3/3 [==============================] - 2s 690ms/step - loss: 1.8982 - accuracy: 0.4167 - val_loss: 0.5469 - val_accuracy: 0.8333
Epoch 6/150
3/3 [==============================] - 2s 714ms/step - loss: 1.4948 - accuracy: 0.5444 - val_loss: 0.5464 - val_accuracy: 0.8229
Epoch 7/150
3/3 [==============================] - 2s 610ms/step - loss: 1.1253 - accuracy: 0.6222 - val_loss: 0.5666 - val_accuracy: 0.8229
Epoch 8/150
3/3

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Dictionary mapping kelas ke nama furniture
class_indices = train_generator.class_indices
index_to_class = {v: k for k, v in class_indices.items()}

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    class_name = index_to_class[class_index]
    one_hot_encoded = np.zeros(14)
    one_hot_encoded[class_index] = 1

    return one_hot_encoded, class_name

# Penggunaan function predict_image
img_path = '/train/meja01/meja134.JPG'
one_hot, class_name = predict_image(img_path)
print(f"Predicted one-hot: {one_hot}, Class name: {class_name}")


1/1 [==============================] - 0s 19ms/step
Predicted one-hot: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.], Class name: meja01


In [ ]:
# Load model
from tensorflow.keras.models import load_model
loaded_model = load_model('/furniture.h5')

# Function untuk prediksi
def predict_image(img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = loaded_model.predict(img_array)
    class_index = np.argmax(prediction)
    class_name = index_to_class[class_index]
    one_hot_encoded = np.zeros(14)
    one_hot_encoded[class_index] = 1

    return one_hot_encoded, class_name

# Penggunaan function predict_image
img_path = '/train/kursi04/kursi413.JPG'
one_hot, class_name = predict_image(img_path)
print(f"Predicted one-hot: {one_hot}, Class name: {class_name}")


1/1 [==============================] - 0s 212ms/step
Predicted one-hot: [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], Class name: kursi04
